In [1]:
import sys
!{sys.executable} -m pip install pymongo
!{sys.executable} -m pip install pyyaml

In [2]:
# Get Mongo database
from yaml import load
from pymongo import MongoClient
from getsecret import getsecret

client = MongoClient(getsecret("MONGODB_URI"))
db = client[getsecret("DB_NAME")]
import urllib.request as req
import json
accounts = json.loads(req.urlopen("http://localhost:5000/synced_emails").read().decode("utf-8"))
# counter for figures
counter = 0

URLError: <urlopen error [Errno 61] Connection refused>

In [14]:
# Look through errors.
error_types = {"NV": {}}
for name in db.list_collection_names():
    if "error" in name:
        for error in db[name].find({"timestamp":{"$gt": 1534749220482}}):
            #print(name)
            # print(json.loads(error["error"])) 
            try:
                error_obj = json.loads(error["error"])
            except TypeError:
                error_obj = error["error"]
            if "stackTrace" in error_obj:
                stack_trace = error_obj["stackTrace"]
                version = "version"
                if version not in error:
                    version = "NV"
                else:
                    version = error[version]
                    if version not in error_types:
                        error_types[version] = {}
                errors_for_version = error_types[version]
                if stack_trace not in errors_for_version:
                    errors_for_version[stack_trace] = 1
                else:
                    errors_for_version[stack_trace] += 1
            else:
                print(error_obj)
for version in error_types:
    print("VERSION: " + str(version))
    for stack in error_types[version]:
        print("STACKTRACE WITH FREQ " +str(error_types[version][stack]))
        print(stack)
        print("\n\n")

VERSION: NV
STACKTRACE WITH FREQ 1
An uncaught Exception occurred on "main" thread.
java.lang.RuntimeException: Error receiving broadcast Intent { act=android.intent.action.USER_PRESENT flg=0x24200010 } in com.tns.gen.android.content.BroadcastReceiver_AccessibilityService_73_56_@b75ddc2
	at android.app.LoadedApk$ReceiverDispatcher$Args.lambda$getRunnable$0(LoadedApk.java:1401)
	at android.app.-$$Lambda$LoadedApk$ReceiverDispatcher$Args$_BumDX2UKsnxLVrE6UJsJZkotuA.run(Unknown Source:2)
	at android.os.Handler.handleCallback(Handler.java:873)
	at android.os.Handler.dispatchMessage(Handler.java:99)
	at android.os.Looper.loop(Looper.java:193)
	at android.app.ActivityThread.main(ActivityThread.java:6669)
	at java.lang.reflect.Method.invoke(Native Method)
	at com.android.internal.os.RuntimeInit$MethodAndArgsCaller.run(RuntimeInit.java:493)
	at com.android.internal.os.ZygoteInit.main(ZygoteInit.java:858)
Caused by: com.tns.NativeScriptException: 
Calling js method onReceive failed

Error: key:

In [ ]:
# Unfortunately, we're still getting those erroneous logs of huge time spent on "" domain since some updates aren't fully reflected yet. Let's delete them again.
done = False
for account in accounts:
    if not done:
        for user in account["android"]:
            if not done:
                print("USER ---- " + user)
                if len([s for s in db[user + "_sessions"].find({"duration": {"$gt": 60 ** 2 * 24 * 7}})]) > 0:
                    print("Num of sessions: " + str(len([s for s in db[user+"_sessions"].find()])))
                    print("Num of bad sessions: " + str(len([s for s in db[user + "_sessions"].find({"domain": "", "duration": {"$gt": 60 ** 2 * 24}})])))
                    print("VERSIONS" + str({s['version'] for s in db[user + "_sessions"].find({"domain": "", "duration": {"$gt": 60 ** 2 * 24}})}))
                    db[user + "_sessions"].delete_many({"domain": "","duration": {"$gt": 60 ** 2 * 24}})
                    print("After delete: " + str(len([s for s in db[user+"_sessions"].find()])))

